In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [5]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
np.random.seed(0)
translator = CirqTranslater(4, untouchable_blocks = [])

translator.env_qubits = [2]

simplifier = Simplifier(translator)
etas = [0.01, 1.]
minimizer = Minimizer(translator, mode="discrimination", params=etas)

In [7]:
cdb = []
for ind, qubits in list(translator.indexed_cnots.items()):
    cdb.append(gate_template(int(ind), block_id=0))
    con, tar = qubits
    cdb.append(gate_template(int(con + translator.number_of_cnots), param_value = np.random.random()))
    cdb.append(gate_template(int(tar + translator.number_of_cnots + translator.n_qubits), param_value = np.random.random()))
circuit_db = pd.DataFrame(cdb)
c0, cdb0 = translator.give_circuit(x_layer_db(translator))

In [9]:
circuit_db = concatenate_dbs([circuit_db, cdb0])
circuit_db["symbol"] = None
circuit, circuit_db  = translator.give_circuit(circuit_db)

simplifier = Simplifier(translator)
simplifier.relative_rules = [simplifier.rule_1, simplifier.rule_2]

simplified_db, ns = simplifier.reduce_circuit(circuit_db)
ss, simplified_db = translator.give_circuit(simplified_db)

mdif = max_diff(translator , circuit_db, simplified_db)
print("\n")
print(mdif)
print(circuit)
print()
print(ss)

simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7ff7340bbf28>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7ff7340bbf28>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7ff7340bbf28>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7ff7340bbf28>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7ff7340bbf28>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7ff7340bbf28>>
simplified using  <bound method Simplifier.rule_4 of <utilities.simplifier.Simplifier object at 0x7ff7340bbf28>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 0x7ff7340bbf28>>
simplified using  <bound method Simplifier.rule_6 of <utilities.simplifier.Simplifier object at 